# Lab 6 - Threat Hunting with Data Science
**Security Data Science**

Sergio Marchena - 16387

UVG - 2022

In [72]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import json
import random

pd.set_option("display.max_columns", None)

## Parte 1: Filtrado y preprocesamiento 

In [64]:
data = [json.loads(line) for line in open('large_eve.json', 'r')]

### 1. Cargue  la  información  del  archivo  large_even.json  en  una  lista,  muestre  la  cantidad  de registros total (deben ser 746, 909).

In [66]:
print('Hay %d registros.' %len(data))

Hay 746909 registros.


### 2. Debido a que estamos buscando dominios DGA, del total de registros, solamente estamos  interesados  en  los  registros  DNS.  Cargue  únicamente  aquellos  registros  en  cuya  llave  se encuentra “DNS”. 

In [69]:
dns = [d for d in data if d['event_type'] == 'dns']

• Muestre la nueva cantidad de registros filtrados. 

In [70]:
print('Hay %d registros DNS.'%len(dns))

Hay 15749 registros DNS.


• Muestre la información de 2 registros cualesquiera. 

In [73]:
random.sample(dns, 2)

[{'timestamp': '2017-07-22T19:17:02.959608-0500',
  'flow_id': 1448943942542205,
  'pcap_cnt': 2755569,
  'event_type': 'dns',
  'vlan': 180,
  'src_ip': '192.168.207.4',
  'src_port': 53,
  'dest_ip': '192.168.198.62',
  'dest_port': 37056,
  'proto': 'UDP',
  'dns': {'type': 'answer',
   'id': 47561,
   'rcode': 'NXDOMAIN',
   'rrname': 'wpad.home'}},
 {'timestamp': '2017-07-22T18:29:17.959205-0500',
  'flow_id': 1845878189826540,
  'pcap_cnt': 505792,
  'event_type': 'dns',
  'vlan': 150,
  'src_ip': '192.168.207.4',
  'src_port': 53,
  'dest_ip': '192.168.205.188',
  'dest_port': 37872,
  'proto': 'UDP',
  'dns': {'type': 'answer',
   'id': 25789,
   'rcode': 'NXDOMAIN',
   'rrname': 'mirrors.tummy.com'}}]

• Debido a que la data consiste en json anidados, utilice la característica json_normalize para 
normalizar la información y asignarla en un dataframe. 

In [84]:
df = pd.json_normalize(dns)
df

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15744,2017-07-22T19:47:03.373435-0500,891876064866556,4441474,dns,120,192.168.207.4,53,192.168.202.172,60654,UDP,answer,33107,en-us.start3.mozilla.com,NaN,NaN,NXDOMAIN,NaN,NaN
15745,2017-07-22T19:47:03.373435-0500,891876064866556,4441474,dns,120,192.168.207.4,53,192.168.202.172,60654,UDP,answer,33107,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN
15746,2017-07-22T19:47:38.146503-0500,2199315651640391,4452917,dns,130,192.168.203.67,50975,192.168.207.4,53,UDP,query,35892,whitecell.localdomain,AAAA,0.0,NaN,NaN,NaN
15747,2017-07-22T19:47:38.146953-0500,2199315651640391,4452918,dns,130,192.168.207.4,53,192.168.203.67,50975,UDP,answer,35892,whitecell.localdomain,NaN,NaN,NXDOMAIN,NaN,NaN


• Como  estamos  buscando  dominios  DGA,  debemos  filtrar  los  registros  DNS  para  aquellos 
registros tipo A (son aquellos que mantienen una dirección IP asociada a un dominio), se debió 
bajar la cantidad a 2849 registros. 

In [77]:
dnsA = df[df['dns.rrtype'] == 'A']
print('Hay %d registros DNS.'%len(dnsA))

Hay 2849 registros DNS.


• Filtre los dominios únicos. 

In [82]:
dnsA = dnsA.drop_duplicates()
dnsA

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
5,2017-07-22T17:33:36.672785-0500,237919524635665,55496,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,41663,2001:07fd:0000:0000:0000:0000:0000:0001,53,UDP,query,45082,api.wunderground.com,A,0.0,NaN,NaN,NaN
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15713,2017-07-22T19:37:54.912593-0500,929379683331281,4301083,dns,120,192.168.202.66,44872,192.168.207.4,53,UDP,query,15886,secure.informaction.com.hsd1.pa.comcast.net,A,0.0,NaN,NaN,NaN
15716,2017-07-22T19:39:02.426497-0500,572502265201153,4319593,dns,180,192.168.198.58,42157,192.168.207.4,53,UDP,query,33079,192.168.22.201:.stayonline.net,A,0.0,NaN,NaN,NaN
15725,2017-07-22T19:42:21.167769-0500,1621384831536985,4374772,dns,150,192.168.205.170,11886,192.168.207.4,53,UDP,query,13068,wpad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
15737,2017-07-22T19:42:44.728139-0500,1137256119409739,4380897,dns,120,192.168.202.172,50164,192.168.207.4,53,UDP,query,59436,client-software.real.com,A,0.0,NaN,NaN,NaN


• Del dataframe de dominios únicos de tipo A, obtenga el TLD (top level domain) utilizando Flare 
como una columna nueva. 

In [86]:
!pip install flare

  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for flare, since package 'wheel' is not installed.
  Running setup.py install for flare ... done


In [89]:
import flare

In [92]:
# Domain TLD Extract
flare.data_science.features.domain_tld_extract('longsubdomain.huntoperator.com')


AttributeError: module 'flare' has no attribute 'data_science'